In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import os

# Configure pararmters

In [2]:
metaPath = "../../metadata/Flickr_FL_Predictions_2014_2019_CLIP-BRF-ZS_1kmGrids.csv"
predLabel = "CLIP-BRF-ZS"
years_to_filter = [2014, 2015, 2016, 2017, 2018, 2019]

In [3]:
#The rule for differernt shape layers.

config = {
    "StateServerData": [
        ["SITE_NAME.notna()"] #State Park #SITE_NAME #First
    ],
    "NationalServerData": [
        ["PARKNAME.notna()", #National Park #PARKNAME #Second
         "SITE_NAME.isna()"] #State Park #SITE_NAME
    ],
    "ParkServerData": {
        "rules": [
            ["NAME20.notna()",  #Urban greenspaces #Urban Area #NAME20
             "Park_Name.notna()", #Park Server #Park_Name  
             "PARKNAME.isna()", #National Park #PARKNAM
             "SITE_NAME.isna()"], #State Park #SITE_NAME
            ["NAME20.isna()",  #Non-urban greenspace, but protect area be priority. #Urban Area #NAME20
             "Park_Name.notna()", #Park Server #Park_Name
             "IUCN_Cat.isna()", #Protected Area #IUCN_Cat
             "PARKNAME.isna()", #National Park #PARKNAME
             "SITE_NAME.isna()"] #State Park #SITE_NAME
        ],
        "subsplits": {
            "Urban_ParkServerData": ["NAME20.notna()"], #Urban_green space
            "OutUrban_ParkServerData": ["NAME20.isna()"] #Outside of Urban_green space
        }
    },
    "ProtectedArealData": [
        ["NAME20.notna()",  #Protected area in urban, but greenspace (park) be priority). #Urban Area #NAME20
         "Park_Name.isna()", #Park Server #Park_Name
         "PARKNAME.isna()", #National Park #PARKNAME
         "SITE_NAME.isna()", #State Park #SITE_NAME
         "IUCN_Cat.notna()"],#Protected Area #IUCN_Cat
        ["NAME20.isna()", #Protecte area in non-urban. #Urban Area #NAME20
         "PARKNAME.isna()", #National Park #PARKNAME
         "SITE_NAME.isna()", #State Park #SITE_NAME
         "IUCN_Cat.notna()"] #Protected Area #IUCN_Cat
    ],
    "OtherAreaData": [
        ["NAME20.isna()", #Urban Area #NAME20
         "Park_Name.isna()", #Park Server #Park_Name
         "PARKNAME.isna()", #National Park #PARKNAME
         "SITE_NAME.isna()", #State Park #SITE_NAME
         "IUCN_Cat.isna()"] ,#Protected Area #IUCN_Cat
    ]
}

# The categories for pie (must match keys in your subsets dict).
categories = [
    "Urban_ParkServerData",
    "OutUrban_ParkServerData",
    "StateServerData",
    "NationalServerData",
    "ProtectedArealData",
    "OtherAreaData"
]

count_columns = [
    "Urban greenspaces",
    "Non-urban greenspaces",
    "State parks",
    "National parks",
    "Protected areas",
    "Other areas"
]

# List of class names
class_names = ["Biking", "Boating", "Camping", "Fishing", "Hiking", "Horseback_Riding",
               "Hunting", "Shelling", "Surfing", "Swimming", "Wildlife_Viewing",
               "Landscape_Aesthetics", "Other"]

# NLCD class mapping
nlcd_class_map = {
    11: "Water", 12: "Water", 21: "Developed", 22: "Developed",
    23: "Developed", 24: "Developed", 31: "Barren", 41: "Forest",
    42: "Forest", 43: "Forest", 51: "Shrubland", 52: "Shrubland",
    71: "Herbaceous", 72: "Herbaceous", 73: "Herbaceous", 74: "Herbaceous",
    81: "Pasture/Hay", 82: "Cultivated Crops", 90: "Wetlands", 95: "Wetlands"
}

#remove class
rm_class = "Other"

# Data preprocessing

In [4]:
metadata = pd.read_csv(metaPath)

# keep only selected years
metadata["takendate"] = pd.to_datetime(metadata["takendate"], errors="coerce")
metadata = metadata[metadata.takendate.dt.year.isin(years_to_filter)]

# build GeoDataFrame with correct CRS (WGS84 degrees)
geo_df = gpd.GeoDataFrame(
    metadata,
    geometry=gpd.points_from_xy(metadata.longitude, metadata.latitude),
    crs="EPSG:4326"
)

In [5]:
metadata.columns, len(metadata)

(Index(['ownerid', 'latitude', 'longitude', 'takendate', 'SITE_NAME',
        'PARKNAME', 'Park_Name', 'IUCN_Cat', 'NAME20', 'NLCD_2023',
        'CLIP-BRF-ZS', 'index_right', 'id', 'left', 'top', 'right', 'bottom',
        'PUD_ID', 'ownerid_ID'],
       dtype='object'),
 590378)

In [6]:
geo_df[predLabel].value_counts()

CLIP-BRF-ZS
Other                   279605
Wildlife_Viewing        172154
Landscape_Aesthetics     67133
Boating                  15491
Swimming                 14463
Shelling                 11575
Fishing                   6677
Hiking                    6125
Surfing                   5000
Hunting                   3603
Biking                    3360
Horseback_Riding          2757
Camping                   2435
Name: count, dtype: int64

In [7]:
#PUDs amount
metadata_LA = geo_df[geo_df[predLabel] == 'Landscape_Aesthetics'].drop_duplicates(subset=['PUD_ID'])
metadata_WL = geo_df[geo_df[predLabel] == 'Wildlife_Viewing'].drop_duplicates(subset=['PUD_ID'])
metadata_HuACT = geo_df[~geo_df[predLabel].isin(['Wildlife_Viewing', 'Landscape_Aesthetics', 'Other'])].drop_duplicates(subset=['PUD_ID'])
metadata_NoCES = geo_df[geo_df[predLabel] == 'Other'].drop_duplicates(subset=['PUD_ID'])

print(len(metadata_LA), len(metadata_WL), len(metadata_HuACT), len(metadata_NoCES))

27790 31139 22578 50381


In [8]:
#PUDs amount in specific years
metadata_LA = metadata_LA[metadata_LA['takendate'].dt.year.isin(years_to_filter)]
metadata_WL = metadata_WL[metadata_WL['takendate'].dt.year.isin(years_to_filter)]
metadata_HuACT = metadata_HuACT[metadata_HuACT['takendate'].dt.year.isin(years_to_filter)]
metadata_NoCES = metadata_NoCES[metadata_NoCES['takendate'].dt.year.isin(years_to_filter)]

print(len(metadata_LA), len(metadata_WL), len(metadata_HuACT), len(metadata_NoCES))
print(len(metadata_LA)+len(metadata_WL)+len(metadata_HuACT)+len(metadata_NoCES))

27790 31139 22578 50381
131888


In [9]:
#Actual amount
metadata_LA_acutal = geo_df[geo_df[predLabel] == 'Landscape_Aesthetics']
metadata_WL_acutal = geo_df[geo_df[predLabel] == 'Wildlife_Viewing']
metadata_HuACT_acutal = geo_df[~geo_df[predLabel].isin(['Wildlife_Viewing', 'Landscape_Aesthetics', 'Other'])]
metadata_NoCES_acutal = geo_df[geo_df[predLabel] == 'Other']

print(len(metadata_LA_acutal), len(metadata_WL_acutal), len(metadata_HuACT_acutal), len(metadata_NoCES_acutal))
print(len(metadata_LA_acutal)+len(metadata_WL_acutal)+len(metadata_HuACT_acutal)+len(metadata_NoCES_acutal))
print(len(metadata_LA_acutal)+len(metadata_WL_acutal)+len(metadata_HuACT_acutal))

67133 172154 71486 279605
590378
310773


In [10]:
def build_condition(df, cond_str):
    col, method = cond_str.split(".")
    method = method.replace("()", "")  # strip parentheses if present
    return getattr(df[col], method)()

def combine_conditions(df, rule_blocks):
    """Combine AND inside a block, OR across blocks"""
    or_blocks = []
    for and_block in rule_blocks:
        conds = [build_condition(df, c) for c in and_block]
        and_cond = conds[0]
        for c in conds[1:]:
            and_cond &= c
        or_blocks.append(and_cond)
    final_cond = or_blocks[0]
    for ob in or_blocks[1:]:
        final_cond |= ob
    return final_cond

def apply_config(df, config):
    results = {}
    for name, rule_def in config.items():
        # allow both old format (list) and dict format (with subsplits)
        if isinstance(rule_def, dict):
            main_rules = rule_def["rules"]
            subsplits = rule_def.get("subsplits", {})
        else:
            main_rules = rule_def
            subsplits = {}

        # main subset
        cond = combine_conditions(df, main_rules)
        subset = df[cond]
        results[name] = subset

        # subsplits (optional)
        for sub_name, sub_rule in subsplits.items():
            sub_cond = combine_conditions(subset, [sub_rule])  # treat single split as one block
            results[sub_name] = subset[sub_cond]
    return results

subsets = apply_config(geo_df, config)

for name, df in subsets.items():
    print(name, len(df))
print("Total:", sum(len(df) for df in subsets.values()))



StateServerData 31642
NationalServerData 46148
ParkServerData 164732
Urban_ParkServerData 155840
OutUrban_ParkServerData 8892
ProtectedArealData 163862
OtherAreaData 183994
Total: 755110


In [11]:
import pandas as pd

results = []

for name, df in subsets.items():
    # --- Compute totals and averages --- #No count for other
    total_puds_id = df[df[predLabel] != rm_class].drop_duplicates(subset=['PUD_ID'])['id'].value_counts()
    avg_pudds_id = (
        total_puds_id / len(years_to_filter)
        if len(years_to_filter) > 0 else pd.Series(dtype=float)
    )

    total_owner_id = df[df[predLabel] != rm_class].drop_duplicates(subset=['ownerid_ID'])['id'].value_counts()
    avg_puds_des_id = (
        total_puds_id / total_owner_id / len(years_to_filter)
        if len(years_to_filter) > 0 else pd.Series(dtype=float)
    )

    # --- Collect summary stats ---
    results.append({
        "Subset": name,
        "avg_puds_sum": avg_pudds_id.sum(),
        "avg_puds_min": avg_pudds_id.min(),
        "avg_puds_max": avg_pudds_id.max(),
        "avg_puds_mean": avg_pudds_id.mean(),
        "avg_puds_std": avg_pudds_id.std(),
        "avg_puds_cv": avg_pudds_id.std()/avg_pudds_id.mean(),
        "avg_puds_des_sum": avg_puds_des_id.sum(),
        "avg_puds_des_min": avg_puds_des_id.min(),
        "avg_puds_des_max": avg_puds_des_id.max(),
        "avg_puds_des_mean": avg_puds_des_id.mean(),
        "avg_puds_des_std": avg_puds_des_id.std(),
        "avg_puds_des_cv": avg_puds_des_id.std()/avg_puds_des_id.mean()
    })

# --- Convert to DataFrame --
results_df = pd.DataFrame(results)

# --- Save to Excel ---
results_df.to_excel("../../metadata/zonal_stats_results.xlsx", index=False)
